# TensorFlow Tutorial 2: TensorBoard
#### MNIST


## Introducción

Código base para generar una CNN. El objetivo es lograr la clasificación de las imágenes de MNIST.

## Dependencias

In [2]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
print ("ready")

ready


Este código usa Python 2.7 en ambiente Anaconda (debería se compatible con cualquier versión de python) y TensorFlow versión:

In [3]:
tf.__version__

'1.3.0'

## Configuración por defecto de la red convolucional

La configuración macro de la red se muestra a continuación. Usted es libre de modificar estos parámetros según estime conveniente.

In [4]:
# Convolutional Layer 1.
filter_size1 = 5          # Filtros son de 5 x 5 pixeles.
num_filters1 = 16         # Hay 16 de estos filtros. Probar con 32

# Convolutional Layer 2.
filter_size2 = 5          # Filtros son de 5x5 pixeles.
num_filters2 = 36         # Hay 16 de estos filtros. Probar con 64

# Fully-connected layer.
fc_size = 128             # Número de neuronas de la capa fully-connected. Probar con 1024

print ("ready")

ready


## Descarga y/o carga de la base de datos

La base de datos MNIST pesa aproximadamente 12 MB. Si no está en el directorio 'data/MNIST', será descargada automáticamente al ejecutar el siguiente bloque de código.

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)


Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


A continuación la base de datos es dividida en tres subconjuntos mutuamente excluyentes.

In [6]:
print("Tamaños de los subconjuntos de la base de datos:")
print("Training-set:\t\t{}".format(len(data.train.labels)))
print("Test-set:\t\t{}".format(len(data.test.labels)))
print("Validation-set:\t\t{}".format(len(data.validation.labels)))

Tamaños de los subconjuntos de la base de datos:
Training-set:		55000
Test-set:		10000
Validation-set:		5000


Las etiquetas de las clases están codificadas en "One-Hot". El dígito "cero" corresponde a la clase 0, el dígito "uno" a la clase 1 y así sucesivamente.Para medir el desempeño de la red se guardan previamente las etiquetas de las clases del conjunto de prueba y validación:

In [7]:
data.test.cls = np.argmax(data.test.labels, axis=1)
data.validation.cls = np.argmax(data.test.labels, axis=1)
print ("ready")

ready


## Dimensiones de los datos

A continuación se definen variables que caracterizan a las imágenes de la base de datos.

In [8]:
# Las imágenes de MNIST son de 28 x 28 pixeles.
img_size = 28
# Tamaño de arreglos unidimensionales que podrían guardar los datos de estas imágenes.
img_size_flat = img_size * img_size
# Tupla que sirve para redimensionar arreglos.
img_shape = (img_size, img_size)
# Número de canales de color de las imágenes. Si las imágenes fueran a color, este número sería 3.
num_channels = 1
# Número de clases.
num_classes = 10
print ("ready")

ready


## TensorFlow Graph

El propósito de TensorFlow es tener un "computational graph" que puede ser ejecutado de forma muy eficiente.

Un "TensorFlow graph" posee las siguientes componentes:

* "Placeholder variables", usadas para entregarle información de entrada al grafo.
* Variables que serán optimizadas para que la CNN se desempeñe mejor.
* Método de optimización para actualizar las variables.
* Función de costo que sirve para encauzar la actualización de variables.
* Formulas matemáticas que use la CNN.


### Sumarios he histogramas

Se creará una función para guardar muchos datos en tensor board

In [9]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

### Funciones para crear variables nuevas

Sirven para que usted cree pesos y biases para su CNN.

Los pesos estan formados por un tensor 4-D con las siguientes dimensiones:

1. Alto del filtro.
2. Ancho del filtro.
3. Canales del filtro.
4. Cantidad de filtros existentes por capa (número de featureMaps a generar).

In [10]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))
print ("ready")

ready


### Función para crear una nueva CNN

Esta función crea una capa convolucional en el "computational graph" de Tensorflow.

Se asume que la entrada es un tensor 4-D con las siguientes dimensiones:

1. Número de imágenes.
2. Dimensión Y (cartesiana) de cada imagen.
3. Dimensión X (cartesiana) de cada imagen.
4. Canales de cada imagen.

La salida es otro tensor 4-D con las siguientes dimensiones:

1. Número de imágenes, el mismo que el de la entrada.
2. Dimensión Y (cartesiana) de cada imagen. 
3. Dimensión X (cartesiana) de cada imagen.
4. Canales producidos por los filtros convolucionales.

In [11]:
def new_conv_layer(input,              # Capa anterior.
                   num_input_channels, # Numero de canales de la capa anterior.
                   filter_size,        # Ancho y alto de cada filtro.
                   num_filters,        # Número de filtros.
                   layer_name = "conv_layer", #nombre de capa
                   use_pooling=True):  # Usar 2x2 max-pooling.
    
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # Forma de los filtros convolucionales (de acuerdo a la API de TF).
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Creación de los filtros.
        with tf.name_scope('weights'):
            weights = new_weights(shape=shape)
            #variable_summaries(weights)
        
        # Creación de biases, uno por filtro.
        with tf.name_scope('biases'):
            biases = new_biases(length=num_filters)
            #variable_summaries(biases)

        # Creación de la operación de convolución para TensorFlow.
        # Notar que se han configurado los strides en 1 para todas las dimensiones.
        # El primero y último stride siempre deben ser uno.
        # Si strides=[1, 2, 2, 1], entonces el filtro es movido
        # de 2 en 2 pixeles a lo largo de los ejes x e y de la imagen.
        # padding='SAME' significa que la imagen de entrada se rellena
        # con ceros para que el tamaño de la salida se mantenga.
        # La otra alternativa es 'VALID', donde no hay zero-padding
        with tf.name_scope('convol'):
            layer = tf.nn.conv2d(input=input,
                                 filter=weights,
                                 strides=[1, 1, 1, 1],
                                 padding='SAME')

            # Agregar los biases a los resultados de la convolución.
            layer += biases
            #tf.summary.histogram('pre_activations', layer)
            
        # Rectified Linear Unit (ReLU).
        layer = tf.nn.relu(layer, name='activation')
        #tf.summary.histogram('activations', layer)

        # Usar pooling para hacer down-sample de la entrada.
        if use_pooling:
            # Este es 2x2 max pooling, lo que significa que se considera
            # una ventana de 2x2 y se selecciona el valor mayor
            # de los 4 pixeles seleccionados. ksize representa las dimensiones de 
            # la ventana de pooling y el stride define cómo la ventana se mueve por la imagen.
            with tf.name_scope('max_pool'):
                layer = tf.nn.max_pool(value=layer,
                                       ksize=[1, 2, 2, 1],
                                       strides=[1, 2, 2, 1],
                                       padding='SAME')



        # La función retorna el resultado de la capa y los pesos aprendidos.
        return layer, weights
print ("ready")

ready


### Función para estirar un tensor de salida

Se usa para reducir las dimensiones del tensor de salida de la capa convolucional a uno 2D que sirva de entrada a la capa fully connected.

In [12]:
def flatten_layer(layer):
    with tf.name_scope('flatten_stage'):
        # Obtener dimensiones de la entrada.
        layer_shape = layer.get_shape()

        # Obtener numero de características.
        num_features = layer_shape[1:4].num_elements()

        # Redimensionar la salida a [num_images, num_features].
        layer_flat = tf.reshape(layer, [-1, num_features])

        # Las dimensiones de la salida son ahora:
        # [num_images, img_height * img_width * num_channels]
        # Retornar
        return layer_flat, num_features
print ("ready")

ready


### Función para crear capa fully-connected

In [13]:
def new_fc_layer(input,          # Capa anterior.
                 num_inputs,     # Numero de entradas.
                 num_outputs,    # Numero de salidas.
                 keep=1,           # probabilidad de permanencia de la neurona
                 layer_name="fc_later",  #nombre de capa
                 use_dropout=False, # usar o no Dropout
                 use_relu=True): # Decide si usar ReLU o no.
    
    with tf.name_scope(layer_name):
        # Crear pesos y biases.
        with tf.name_scope('weights'):
            weights = new_weights(shape=[num_inputs, num_outputs])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = new_biases(length=num_outputs)
            variable_summaries(biases)
            
        # Evaluar capa fully connected.
        layer = tf.matmul(input, weights) + biases
        tf.summary.histogram('pre_activations', layer)

        # Usar ReLU? OJO: última capa no utiliza activación, debido a normalización softmax
        if use_relu:
            layer = tf.nn.relu(layer, name='activation')
            tf.summary.histogram('activations', layer)

        # Usar DropOut? OJO: última capa no usa DropOut, osino perdería codificación OneHot
        if use_dropout:
            with tf.name_scope('dropout'):
                layer=tf.nn.dropout(layer,keep_prob=keep)

        return layer
print ("ready")

ready


### Placeholder variables

Las variables "placeholder" sirven como entradas para el "computational graph" de Tensorflow.
Primero se define una variable placeholder para las imágenes de entrada. Estas son interpretadas como "tensores" (vectores o matrices multidimensionales). El tipo de datos se configura como `float32`, y su forma se deja como `[None, img_size_flat]`, donde `None` significa que el tensor puede contener un numero arbitrario de imágenes, cada una representada como un vector de largo `img_size_flat`.

In [14]:
with tf.name_scope('input'):
    keep_prob = tf.placeholder(tf.float32)
    
    x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x-input')
    x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

    y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true-input')
    y_true_cls = tf.argmax(y_true, axis=1)
print ("ready")

ready


### Capa de convolución 1

Creación de la primera capa de convolución. Al final se realiza submuestreo con un 2x2 max-pooling.

In [15]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   layer_name="conv_layer_1",
                   use_pooling=True)

layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   layer_name="conv_layer_2",
                   use_pooling=True)
    
layer_flat, num_features = flatten_layer(layer_conv2)

layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         keep=keep_prob,
                         layer_name="fc_layer_1",
                         use_dropout=True,
                         use_relu=True)

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         layer_name="fc_layer_2",
                         use_relu=False)
print ("ready")

ready


### Clase predicha

Se utiliza softmax para normalizar la salida, luego se toma el valor máximo.

### Función de costo

Usamos cross-entropy.
Tensorflow la implementa de forma nativa. Como la función calcula softmax internamente, debe entregarsele la salida de la capa fully-connected 2 directamente.

In [16]:
with tf.name_scope('soft-max'):
    y_pred = tf.nn.softmax(layer_fc2)
with tf.name_scope('predicted_class'):
    y_pred_cls = tf.argmax(y_pred, axis=1)
print("ready")

ready


In [17]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                            labels=y_true)
    with tf.name_scope('cost'):
        cost = tf.reduce_mean(cross_entropy)


### Método de optimización

Usamos `AdamOptimizer`.
Puede cambiarlo según se pida en la tarea.

In [18]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
print("ready")

ready


### Medida de desempeño

Se obtiene un vector de booleanos que indican si la clase predicha es o no igual a la clase verdadera de cada imagen.
Luego dicho vector se niega para que True sea 1, y se calcula el promedio.

In [20]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)
print("ready")

ready


## Ejecutar TensorFlow 

### Crear TensorFlow session

Ya creado el grafo de TF, se crea una sesión para ejecutarlo.

In [21]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)) #tf.InteractiveSession() #with tf.Session() as sess:
print("ready")

ready


### Logs de TensorBoard

Después de ejecutar el entrenamiento, abrir una consola con el ambiente de TensorFlow activo y ejecutar

tensorboard --logdir="/home/asceta/Documents/tfTutorial/TB/tmp/mnist_tutorial/2"

A continuación se generará un link para abrir en el navegador, el que mostrará la info del grafo en TensorBoard

In [22]:
# path a logs donde se guardará la info del grafo. Cambiar por ruta propia.
LOGDIR = "/home/asceta/Documents/tfTutorial/TB/tmp/mnist_tutorial/2"

#recolección de todos los summaries
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(LOGDIR + '/train', sess.graph)
val_writer = tf.summary.FileWriter(LOGDIR + '/val')

### Inicialización de variables

In [23]:
sess.run(tf.global_variables_initializer())
print("ready")

ready


### Función para realizar optimización

In [24]:
# Entrenamiento realizado por batches.
train_batch_size = 100

# Contador de iteraciones.
total_iterations = 0

#entregar variables para entrenamiento y validación
def feed_dict(mnist, train, batch_size, dropout=1):
    if train:
        xs, ys = data.train.next_batch(batch_size)
        k = dropout
    else:
        xs= data.validation.images
        ys = data.validation.labels
        k = 1.0
    return xs, ys, k


def optimize(num_iterations):

    #contador de epocas
    j=0
    #numero de iteraciones por epoca
    ep=55000/train_batch_size
    
    # Tiempo de inicio
    start_time = time.time()

    for i in range(num_iterations):

        # Obtener batch de conjunto de entrenamiento.
        x_batch, y_true_batch, keep = feed_dict(data, True, train_batch_size, dropout=0.5)

        # Se pone el batch en un diccionario asignandole nombres de las
        # variables placeholder antes definidas.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch, keep_prob: keep}



        # Se imprime numero de epoca de entrenamiento, junto al accuracy de Validación.
        if i % ep == 0:
            msg = "epoca: {0:>6}"
            print(msg.format(j+1))
            
            d_t = feed_dict(data, False, train_batch_size)
            inp_t = {x:d_t[0], y_true:d_t[1], keep_prob:d_t[2]}
            summary, acc = sess.run([merged, accuracy], feed_dict=inp_t) 
            val_writer.add_summary(summary, i)
            num_val = len(data.validation.images)
            correct_sum = acc*num_val
            msg = "Accuracy on Validation-Set: {0:.1%} ({1} / {2})"
            print(msg.format(acc, correct_sum, num_val))
            j+=1
        
        #se imprime el train accuracy cada ep/10 iteraciones
        if i % (ep/10) == 0:
            train_acc = {x: x_batch,
                           y_true: y_true_batch, keep_prob: 1.0}
            summary, acc = sess.run([merged, accuracy], feed_dict=train_acc)
            train_writer.add_summary(summary, i)
            msg = "Iterations: {0:>6}, Training Accuracy: {1:>6.1%}"
            print(msg.format(i, acc))
        
        # Ejecución del optimizador con los batches del diccionario.
        sess.run(optimizer, feed_dict=feed_dict_train)

    # Tiempo de finalización.
    end_time = time.time()

    # Tiempo transcurrido.
    time_dif = end_time - start_time

    train_writer.close()
    val_writer.close()
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))


print("ready")

ready


### Función para mostrar desempeño en test-set

In [25]:
# Dividir test set en batches. (Usa batches mas pequeños si la RAM falla).
test_batch_size = 256

def print_test_accuracy():

    # Número de imagenes en test-set.
    num_test = len(data.test.images)

    # Crea arreglo para guardar clases predichas.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Calcular clases predichas.
    i = 0
    while i < num_test:
        
        j = min(i + test_batch_size, num_test)
        images = data.test.images[i:j, :]
        labels = data.test.labels[i:j, :]
        feed_dict = {x: images,
                     y_true: labels, keep_prob: 1.0}

        cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)
        i = j
    
    # Labels reales.
    cls_true = data.test.cls

    # Arreglo booleano de clasificaciones correctas.
    correct = (cls_true == cls_pred)
    
    #Número de clasificaciones correctas.
    correct_sum = correct.sum()

    # Accuracy
    acc = float(correct_sum) / num_test
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))
    
print("ready")

ready


## Optimizar

Elige un número de iteraciones y entrena la CNN.

In [26]:
#Definir número de iteraciones que desea entrenar a la red
optimize(num_iterations=1101) 

epoca:      1
Accuracy on Validation-Set: 13.8% (692.00001657 / 5000)
Iterations:      0, Training Accuracy:  11.0%
Iterations:     55, Training Accuracy:  52.0%
Iterations:    110, Training Accuracy:  58.0%
Iterations:    165, Training Accuracy:  76.0%
Iterations:    220, Training Accuracy:  79.0%
Iterations:    275, Training Accuracy:  82.0%
Iterations:    330, Training Accuracy:  91.0%
Iterations:    385, Training Accuracy:  80.0%
Iterations:    440, Training Accuracy:  94.0%
Iterations:    495, Training Accuracy:  96.0%
epoca:      2
Accuracy on Validation-Set: 90.7% (4535.00002623 / 5000)
Iterations:    550, Training Accuracy:  91.0%
Iterations:    605, Training Accuracy:  91.0%
Iterations:    660, Training Accuracy:  87.0%
Iterations:    715, Training Accuracy:  90.0%
Iterations:    770, Training Accuracy:  88.0%
Iterations:    825, Training Accuracy:  92.0%
Iterations:    880, Training Accuracy:  94.0%
Iterations:    935, Training Accuracy:  86.0%
Iterations:    990, Training Ac

In [27]:
print_test_accuracy()

Accuracy on Test-Set: 94.0% (9401 / 10000)


### Cerrar TensorFlow Session

In [223]:
# Si usted ejecuta esta linea de código debe cerrar el notebook y reiniciarlo.
# Es solo para informar como liberar los recursos que ocupa TF.
tf.reset_default_graph()
sess.close()
print("ready")

ready


### Diferencia entre .eval y .run 

In [51]:
t = tf.constant(42.0)
u = tf.constant(37.0)
tu = t*u
ut = u*t
with tf.Session() as sess:
   sess.run(tf.global_variables_initializer())
   assert t.eval() == sess.run(t)
   tu.eval()  # runs one step and useful to evaluate a tensor, equivalent to sess.run(tu)
   ut.eval()  # runs one step
   sess.run([tu, ut])  # evaluates both tensors in a single step